In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.insert(0,"/content/drive/MyDrive/Colab Notebooks/TestSAWO/")

from Rule import Rule
from RuleSet import RuleSet
from Node import Node
from Architecture import Architecture
from SawoNN import SawoNN
from Member import Member

import time
import json
import random
import numpy as np
import tensorflow as tf

folder_path = "/content/drive/MyDrive/Colab Notebooks/TestSAWO/"
dataset_path = folder_path + "datasets/"

rs = RuleSet(folder_path + "grammar.txt")
print(rs)


# CAN CHANGE THE FOLLOWING
dataset = 11
generate_sample = True
# DON'T CHANGE ANYTHING ELSE


class_T_reg_F = True
if dataset >= 9:
    class_T_reg_F = False

if dataset == 1:
    seed = 109031
elif dataset == 2:
    seed = 32228
elif dataset == 3:
    seed = 97483
elif dataset == 4:
    seed = 110457
elif dataset == 5:
    seed = 74949
elif dataset == 6:
    seed = 29240
elif dataset == 7:
    seed = 23745
elif dataset == 8:
    seed = 45647
elif dataset == 9:
    seed = 105953
elif dataset == 10:
    seed = 74080
elif dataset == 11:
    seed = 62741
elif dataset == 12:
    seed = 60320

In [ ]:
#CREATE SAWO SAMPLE

if generate_sample:
    sample_path = folder_path + "samples/"

    sample_size = 30
    num_epochs = 20
    num_batches = 5

    chromosomes = [None]*13
    chromosomes[1] = ['0000000', '0101101', '0001101', '0001101', '0000000', '0001101', '0001101', '0000000', '0011010', '0000000', '0110110', '0000000', '1010111', '0000000', '0101000', '0000000', '0010001', '0000000', '0000001', '0000000', '0001000', '0000000', '0001110', '0000000', '1110011', '0000000', '1101011', '0000000', '1110100', '0100000', '1010000']
    chromosomes[2] = ['0000000', '0001101', '0001001', '0001101', '0000000', '0001001', '0001101', '0000000', '1111100', '0000000', '1110001', '0000000', '0001111', '0000000', '0100110', '0000000', '1110011', '0010000', '0001101', '0000000', '1101010', '0000000', '0001100', '0000000', '1100011', '0000000', '1010011', '0000000', '0011001', '0000000', '1001110']
    chromosomes[3] = ['1001000', '0011101', '0001101', '0001000', '1001000', '0001101', '0001000', '0000000', '0000011', '0000000', '0000101', '0000000', '1100110', '0000000', '0000001', '0001010', '1001110', '0000000', '0110100', '0000000', '1010011', '0000000', '1010001', '0000000', '1000110', '0001000', '0001101', '0000000', '0111000', '0000000', '0000110']
    chromosomes[4] = ['0000001', '0001001', '0001101', '0001101', '0000001', '0001101', '0001101', '0000000', '0101000', '0000000', '1000011', '0000000', '0110110', '1000000', '1110101', '0000000', '0111101', '0000000', '0110100', '0000010', '1100011', '0000000', '1010011', '0000000', '1110010', '0000000', '0010100', '0000000', '0100001', '0000000', '1011101']
    chromosomes[5] = ['0000000', '0001101', '0001101', '0001101', '0000000', '0001101', '0001101', '0000000', '0111000', '0000000', '0010010', '0000000', '1111011', '0000000', '0101000', '0000000', '0010001', '0000000', '1100001', '0000000', '0010111', '0000000', '0001000', '0000000', '1101111', '0000010', '0111000', '0000000', '1101111', '0000000', '1010000']
    chromosomes[6] = ['0000000', '0001111', '0001111', '0001101', '0000000', '0001111', '0001101', '0000000', '0110110', '0000000', '0001010', '0000000', '0100010', '0000000', '0101011', '0000100', '0010101', '0000000', '1011101', '0000000', '0100000', '0000000', '1111110', '0000000', '0100111', '0000000', '1100001', '0100000', '1011111', '0000000', '0010001']
    chromosomes[7] = ['0010000', '0001101', '0001101', '0001101', '0010000', '0001101', '0001101', '0000000', '1110101', '0000000', '0110110', '0000000', '0111101', '0000000', '1110100', '0000000', '1001010', '0000000', '0010011', '0000000', '1101010', '0000000', '1110101', '0000000', '0111011', '0000000', '0010011', '0100000', '0100000', '0000000', '1011100']
    chromosomes[8] = ['0000000', '0001101', '0001101', '0001101', '0000000', '0001101', '0001101', '0000000', '0100000', '0000000', '0011100', '0000000', '1001101', '0000000', '1101101', '0000000', '0101011', '0001000', '0000001', '0000000', '0000111', '0000000', '0100000', '0000000', '1111101', '0000010', '1111000', '0000000', '0111010', '0000000', '0101111']
    chromosomes[9] = ['0000000', '0001101', '0001101', '0001101', '0000000', '0001101', '0001101', '0000000', '0000101', '0000000', '0101101', '0000000', '0100011', '0000000', '0011000', '0000000', '0000011', '0000000', '0011001', '0000000', '0001101', '0000000', '1011011', '0100000', '1010111', '0000000', '0101010', '0000000', '0001101', '0000000', '0001010']
    chromosomes[10] = ['0000000', '0001101', '0001111', '0001101', '0000000', '0001111', '0001101', '0000000', '0000110', '0000000', '1000001', '0000100', '0000001', '0000000', '1001111', '0000000', '1011010', '0000000', '0011100', '0001000', '0110111', '0000000', '1010010', '0000000', '1010000', '0000000', '0110011', '0000000', '0001001', '0000000', '0000100']
    chromosomes[11] = ['0000000', '0001101', '0001101', '0001101', '0000000', '0001101', '0001101', '0000000', '0011101', '0000000', '0001010', '0010000', '0000010', '0000000', '0010000', '0100000', '1011010', '0000010', '0000111', '0000000', '0010100', '0000000', '0001000', '0010000', '1001111', '0000000', '0011010', '1000000', '0110100', '0000000', '1000001']
    chromosomes[12] = ['0000000', '0001101', '1001101', '0001101', '0000000', '1001101', '0001101', '0100000', '0100111', '0000010', '1001110', '0000010', '0101100', '0000000', '0000101', '0000000', '0000111', '0000000', '0001000', '0001000', '0011011', '1000000', '0010101', '0000000', '0101011', '0000000', '0101101', '0000000', '0001001', '0000000', '0010110']

    chromosome = chromosomes[dataset]

    chromosome_int_version = [int(x, 2) for x in chromosome]
    hash = sum(chromosome_int_version)
    sample_filename = str(dataset) + "_" + "_" + str(hash) + ".json"


    print("dataset:", dataset)
    print("class_T_reg_F:", class_T_reg_F)
    print("seed:", seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    print("sample_size:", sample_size)
    print("num_epochs:", num_epochs)
    print("num_batches:", num_batches)
    print("chromosome:", chromosome)
    print("sample_filename:", sample_filename)

    from sklearn.model_selection import train_test_split
    X = np.load(dataset_path + str(dataset) + '_input.npy')
    print("X.shape:", X.shape)
    y = np.load(dataset_path + str(dataset) + '_target.npy')
    print("y.shape:", y.shape)

    equation = rs.constructEquation(chromosome)
    print("equation:", equation)
    print()

    meta_obj = {}
    meta_obj["dataset"] = dataset
    meta_obj["class_T_reg_F"] = class_T_reg_F
    meta_obj["seed"] = seed
    meta_obj["sample_size"] = sample_size
    meta_obj["num_epochs"] = num_epochs
    meta_obj["num_batches"] = num_batches
    meta_obj["chromosome"] = chromosome
    meta_obj["sample_filename"] = sample_filename
    meta_obj["equation"] = equation

    start_time = time.time()
    
    sample = []
    for i in range(sample_size):
        print(i)
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=(seed+i))

        best_sawo = SawoNN(equation, X_train, y_train, X_test, y_test)
        essential_eq = best_sawo.printEssentialInfo()
        print(essential_eq) 
        meta_obj["essential_eq"] = essential_eq
        p, e, w = best_sawo.train(num_epochs, num_batches, class_T_reg_F)
        if class_T_reg_F:
            print(max(e), ">", e)
            sample.append(max(e))
        else:
            print(min(e), "<", e)
            sample.append(min(e))
        print()
    
    end_time = time.time()
    
    print("sample:", sample)
    print("avg:", np.mean(sample))
    print("std:", np.std(sample))
    print("min:", min(sample))

    time_taken = (end_time - start_time)
    meta_obj["time_taken"] = time_taken

    sample = [float(x) for x in sample]
    meta_obj["sample"] = sample

    print(meta_obj)

    sample_file = open(sample_path + sample_filename, 'w')
    json.dump(meta_obj, sample_file)
    sample_file.close()

else:

    samples = [None]*13
    samples[1] = [0.9790209531784058, 0.9930070042610168, 0.9790209531784058, 0.9790209531784058, 0.9860140085220337, 0.9790209531784058, 0.9790209531784058, 0.9790209531784058, 0.9930070042610168, 0.9440559148788452, 0.9510489702224731, 0.9650349617004395, 0.9860140085220337, 0.9720279574394226, 0.9860140085220337, 0.9580419659614563, 0.9790209531784058, 0.9230769276618958, 0.9720279574394226, 0.9790209531784058, 0.9720279574394226, 0.9650349617004395, 0.9650349617004395, 0.9790209531784058, 0.9790209531784058, 0.9650349617004395, 0.9720279574394226, 0.9650349617004395, 0.9790209531784058, 0.9580419659614563]
    samples[2] = [0.9857212901115417, 0.9935992360115051, 0.9798129200935364, 0.9842442274093628, 0.9857212901115417, 0.9871984124183655, 0.9867060780525208, 0.9768586754798889, 0.9807976484298706, 0.9901526570320129, 0.9871984124183655, 0.9812899827957153, 0.9857212901115417, 0.9881831407546997, 0.9798129200935364, 0.9871984124183655, 0.9862136840820312, 0.9783357977867126, 0.9768586754798889, 0.9881831407546997, 0.9906449913978577, 0.9901526570320129, 0.9847365617752075, 0.9817823767662048, 0.9867060780525208, 0.9807976484298706, 0.9817823767662048, 0.9827671051025391, 0.9940915703773499, 0.9842442274093628]
    samples[3] = [0.8289473652839661, 0.8684210777282715, 0.7894737124443054, 0.8684210777282715, 0.9210526347160339, 0.8552631735801697, 0.8289473652839661, 0.8289473652839661, 0.9078947305679321, 0.8815789222717285, 0.8815789222717285, 0.8684210777282715, 0.8289473652839661, 0.8684210777282715, 0.9210526347160339, 0.8421052694320679, 0.8684210777282715, 0.8815789222717285, 0.8026315569877625, 0.8026315569877625, 0.8421052694320679, 0.8026315569877625, 0.8421052694320679, 0.8421052694320679, 0.8552631735801697, 0.8157894611358643, 0.8552631735801697, 0.8552631735801697, 0.8421052694320679, 0.8421052694320679]
    samples[4] = [0.9596697092056274, 0.9539536237716675, 0.956017792224884, 0.9503017067909241, 0.9518895149230957, 0.9599872827529907, 0.9615750908851624, 0.960304856300354, 0.960304856300354, 0.9595109820365906, 0.9522070288658142, 0.953159749507904, 0.9515719413757324, 0.9555414319038391, 0.9537948369979858, 0.9593521952629089, 0.9536360502243042, 0.9566528797149658, 0.957923173904419, 0.9587170481681824, 0.9526833891868591, 0.9601460695266724, 0.9550650715827942, 0.9571292400360107, 0.9583994746208191, 0.9561765789985657, 0.9577643871307373, 0.9607812166213989, 0.956017792224884, 0.9428389668464661]
    samples[5] = [0.9473684430122375, 0.8684210777282715, 0.9736841917037964, 0.8157894611358643, 0.8421052694320679, 0.8947368264198303, 0.8947368264198303, 0.8157894611358643, 0.9210526347160339, 0.8421052694320679, 0.8947368264198303, 0.9210526347160339, 0.8684210777282715, 0.9210526347160339, 0.8421052694320679, 0.8947368264198303, 0.8421052694320679, 0.8947368264198303, 0.8684210777282715, 0.8421052694320679, 0.9473684430122375, 0.8947368264198303, 0.8684210777282715, 0.8157894611358643, 0.9210526347160339, 0.9210526347160339, 0.8947368264198303, 0.8684210777282715, 0.9210526347160339, 0.8947368264198303]
    samples[6] = [0.5849999785423279, 0.5899999737739563, 0.6150000095367432, 0.574999988079071, 0.5799999833106995, 0.6025000214576721, 0.5950000286102295, 0.5699999928474426, 0.5799999833106995, 0.5849999785423279, 0.6025000214576721, 0.6299999952316284, 0.5550000071525574, 0.5849999785423279, 0.6150000095367432, 0.5774999856948853, 0.5249999761581421, 0.5824999809265137, 0.5625, 0.5674999952316284, 0.5724999904632568, 0.5699999928474426, 0.5525000095367432, 0.5450000166893005, 0.5475000143051147, 0.5774999856948853, 0.5724999904632568, 0.6000000238418579, 0.5525000095367432, 0.5824999809265137]
    samples[7] = [0.5185185074806213, 0.5925925970077515, 0.6481481194496155, 0.6851851940155029, 0.5185185074806213, 0.5740740895271301, 0.5185185074806213, 0.7222222089767456, 0.5185185074806213, 0.5925925970077515, 0.5185185074806213, 0.5555555820465088, 0.5740740895271301, 0.6296296119689941, 0.5370370149612427, 0.6481481194496155, 0.48148149251937866, 0.6851851940155029, 0.5185185074806213, 0.5925925970077515, 0.5740740895271301, 0.6296296119689941, 0.6111111044883728, 0.6481481194496155, 0.5370370149612427, 0.5925925970077515, 0.5925925970077515, 0.5185185074806213, 0.5740740895271301, 0.5925925970077515]
    samples[8] = [0.9811320900917053, 1.0, 0.9622641801834106, 0.9245283007621765, 0.9811320900917053, 0.9811320900917053, 0.9433962106704712, 0.9433962106704712, 1.0, 0.9622641801834106, 0.9433962106704712, 0.9811320900917053, 0.9245283007621765, 0.9433962106704712, 0.9056603908538818, 0.9811320900917053, 0.8867924809455872, 0.9433962106704712, 0.9811320900917053, 0.9811320900917053, 0.9245283007621765, 0.9622641801834106, 0.9811320900917053, 0.9245283007621765, 1.0, 0.9811320900917053, 0.9622641801834106, 0.9433962106704712, 0.9433962106704712, 0.9433962106704712]
    samples[9] = [0.003910244442522526, 0.003931925166398287, 0.004149009473621845, 0.004939777310937643, 0.006025268230587244, 0.004320293199270964, 0.0032007023692131042, 0.0032279393635690212, 0.00560714490711689, 0.004499190952628851, 0.0041396901942789555, 0.0038924207910895348, 0.00343576748855412, 0.004619924817234278, 0.004885676316916943, 0.004542565438896418, 0.003732542973011732, 0.004452012944966555, 0.002876856131479144, 0.0031794924288988113, 0.003406412899494171, 0.0041904631070792675, 0.005438821390271187, 0.004601324908435345, 0.00391923263669014, 0.004483118187636137, 0.004565812647342682, 0.005096863489598036, 0.0037267329171299934, 0.004665009677410126]
    samples[10] = [0.012395686469972134, 0.01830693706870079, 0.020129794254899025, 0.015355817973613739, 0.014401030726730824, 0.01795993372797966, 0.008753811940550804, 0.015638824552297592, 0.011573820374906063, 0.00993205327540636, 0.012894046492874622, 0.014218657277524471, 0.01327311061322689, 0.012275966815650463, 0.013066870160400867, 0.009632406756281853, 0.012765273451805115, 0.011667943559587002, 0.008482107892632484, 0.02472653053700924, 0.017473148182034492, 0.007742130663245916, 0.01062890887260437, 0.01846466213464737, 0.018208634108304977, 0.011252416297793388, 0.014035405591130257, 0.0084536112844944, 0.0099562993273139, 0.01940983347594738]
    samples[11] = [0.007954437285661697, 0.007609899155795574, 0.008145579136908054, 0.008067002519965172, 0.007648859638720751, 0.007545954547822475, 0.0077608563005924225, 0.008608642034232616, 0.007512789219617844, 0.007306179031729698, 0.007891870103776455, 0.007011138368397951, 0.007741579320281744, 0.008076297119259834, 0.007449021562933922, 0.007832806557416916, 0.008440663106739521, 0.00880969874560833, 0.008151650428771973, 0.007624266669154167, 0.007255620788782835, 0.0087488554418087, 0.008500288240611553, 0.00827966257929802, 0.007494653575122356, 0.009066271595656872, 0.00850632507354021, 0.00817562360316515, 0.007817498408257961, 0.008321051485836506]
    samples[12] = [0.014854172244668007, 0.027886969968676567, 0.014547284692525864, 0.013750430196523666, 0.0226652380079031, 0.021135330200195312, 0.021892596036195755, 0.010649566538631916, 0.015209091827273369, 0.01070213969796896, 0.013910399749875069, 0.013486829586327076, 0.02207252010703087, 0.024296974763274193, 0.013375330716371536, 0.01463235728442669, 0.01338519249111414, 0.024159537628293037, 0.01461845263838768, 0.023595906794071198, 0.015727942809462547, 0.01466539315879345, 0.025551268830895424, 0.01762179657816887, 0.012852721847593784, 0.026703014969825745, 0.015196841210126877, 0.02424931339919567, 0.016019899398088455, 0.01972065679728985]

    sample = samples[dataset]


print("SAWO")
print(sample)
print("avg", stats.mean(sample))
print("std", np.std(sample))
print("min", min(sample))
print("max", max(sample))
print()


In [ ]:
#CREATE BASELINE SAMPLE

random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

learning_rates = [None,0.23,0.18,0.22,0.50,0.48,0.48,0.39,0.22,0.34,0.14,0.18,0.26]
learning_rate = learning_rates[dataset]

base_sample = []
for i in range(sample_size):  
    print(i)

    val = np.NAN

    count = 0
    while count < 2 and np.isnan(val):
        count += 1

        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=(seed+i))

        init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.2)

        model = keras.Sequential([
            keras.Input(shape=(X_train.shape[1])),
            layers.Dense(15, kernel_initializer=init),
            layers.Dense(15, kernel_initializer=init),
            layers.Dense(15, kernel_initializer=init),
            layers.Dense(y_train.shape[1], kernel_initializer=init),
        ])


        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)

        model.compile(loss='mse', metrics = ['accuracy'], optimizer=opt)

        b_s = math.ceil(X_train.shape[0]/num_batches)
        res = model.fit(x=X_train, y=y_train, epochs=num_epochs, batch_size=b_s, validation_data=(X_test, y_test))

        if class_T_reg_F:
            print(res.history['val_accuracy'])
            val = max(res.history['val_accuracy'])
            print(val)
        else:
            print(res.history['val_loss'])
            val = min(res.history['val_loss'])
            print(val)


    if np.isnan(val):
        val = np.inf
    print("->", val)

    
    base_sample.append(val)
    print()


print("BASE")
print(base_sample)
print("avg", stats.mean(base_sample))
print("std", np.std(base_sample))
print("min", min(base_sample))
print("max", max(base_sample))
print()

In [ ]:
from scipy.stats import ttest_rel

print("PAIRED T TEST")
t, p = ttest_rel(base_errors.copy(), best_errors.copy())
print('t=%.10f, p=%.10f' % (t, p))
if p > 0.05:
	print('Probably the same distribution')
else:
	print('Probably different distributions')
 

alpha = 0.5
print("Confidence:", ((1.0 - alpha)*100), "%")


if class_T_reg_F:

    if p / 2 < alpha and t > 0:
        print("BASE > BEST i.e. BASE wins!")
        print("Will win at %.6f" % ((1.0 - (p/2))*100), "% confidence with p/2 =", "%.6f" % (p/2))
    else:
        print("BASE <= BEST i.e. no clear winner")

    if p / 2 < alpha and t < 0:
        print("BASE < BEST i.e. BEST wins!")
        print("Will win at %.6f" % ((1.0 - (p/2))*100), "% confidence with p/2 =", "%.6f" % (p/2))
    else:
        print("BASE >= BEST i.e. no clear winner")

else:

    if p / 2 < alpha and t > 0:
        print("BASE > BEST i.e. BEST wins!")
        print("Will win at %.6f" % ((1.0 - (p/2))*100), "% confidence with p/2 =", "%.6f" % (p/2))
    else:
        print("BASE <= BEST i.e. no clear winner")

    if p / 2 < alpha and t < 0:
        print("BASE < BEST i.e. BASE wins!")
        print("Will win at %.6f" % ((1.0 - (p/2))*100), "% confidence with p/2 =", "%.6f" % (p/2))
    else:
        print("BASE >= BEST i.e. no clear winner")

